In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import sys
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, classification_report, precision_score, recall_score
from sklearn.model_selection import train_test_split

#################################
# from xgboost import XGBClassifier
# from sklearn.ensemble import AdaBoostClassifier

# from sklearn.pipeline import Pipeline
# from sklearn.feature_selection import SelectKBest, f_classif
# from sklearn.datasets import make_classification
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_curve, roc_auc_score
# from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
# from imblearn.over_sampling import SMOTENC
# from imblearn.under_sampling import RandomUnderSampler
# from bayes_opt import BayesianOptimization
# from sklearn.linear_model import LogisticRegression
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
# from sklearn.metrics import average_precision_score, precision_recall_curve, plot_precision_recall_curve

In [ ]:
from google.colab import drive
#drive.flush_and_unmount() #you can unmount your drive with this code
drive.mount('/gdrive', force_remount=True)

In [ ]:
project_dir = "/gdrive/My Drive/Final_Project_CrystalsFirst/Model/"
%cd {project_dir}

# Loading Saved Models


In [ ]:
# import logging
# tf.get_logger().setLevel(logging.ERROR)

# new_model_EffNet = tf.keras.models.load_model('saved_model/EffNet')

# tf.get_logger().setLevel(logging.INFO)

In [ ]:
# new_model_ResNet = tf.keras.models.load_model('saved_model/ResNet')

In [ ]:
new_model_Inception = tf.keras.models.load_model('saved_model/Inception_9')

In [ ]:
new_model_VGG19 = tf.keras.models.load_model('saved_model/VGG19_batch_16_epoch_50')

# Create DataFrame to collect predictions

In [ ]:
### Create DataFrame with test images to be populated by the prediction of each model 

folder_path = "/gdrive/My Drive/Final_Project_CrystalsFirst/Model/labels/test/"
img_files = {}
for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".JPG")):
      img_files[filename] = ()

df_img = pd.DataFrame.from_dict(img_files, orient="index")

In [ ]:
df_img.head(2)

In [ ]:
### import JSON file with true labels
df_json = pd.read_json(project_dir+"source/image_labels.json",
                       orient="columns")

In [ ]:
df_json.head(2)
len(df_json)

In [ ]:
### Merge DataFrames

df_pred = df_img.join(df_json, on=None, how='left')
# df_pred["y_true"] = df_pred["binary_label"]
# df_pred.drop("binary_label", axis=1, inplace=True)
df_pred.head(2)

In [ ]:
len(df_pred)

# Predict and collect in Dataframe

In [ ]:
IMG_SIZE = 229
classes = ['crystal', 'no_crystal']
folder_path = "/gdrive/My Drive/Final_Project_CrystalsFirst/Model/labels/test/"

# models = [new_model_ResNet, new_model_EffNet, new_model_Inception , new_model_VGG19]
models = [new_model_Inception]

# models_names = ["ResNet", "EffNet", "Inception", "VGG19"]
models_names = ["Inception"]

i=0
for new_model in models:
  predictions = {}
  for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".JPG")):
        img = keras.preprocessing.image.load_img(folder_path+filename, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = keras.preprocessing.image.img_to_array(img)/255
        img_array = tf.expand_dims(img_array, 0)    
        pred = new_model.predict(img_array)
        predictions[filename] = (classes[np.argmax(pred)], (np.max(pred)))
  # df_pred[models_names[i]+ "_file"]   = predictions.keys()
  df_pred[models_names[i] + "_label"] = ([t[0] for t in predictions.values()])
  df_pred[models_names[i] + "_pred"]  = ([t[1] for t in predictions.values()])
  i=i+1

In [ ]:
df_pred[df_pred['y_true']!=df_pred['Inception_label']]

In [ ]:
df_pred[df_pred['y_true']!=df_pred['VGG19_label']]

# Train / Test split

In [ ]:
X = df_pred.iloc[:,1:]
Y = df_pred.iloc[:,0]

X_train,X_test,y_train,y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
X_train.head(2)

In [ ]:
y_train.head(2)

## Adjust probabilities of wrongly classified images

In [ ]:
# only apply to training set
# col_label = X_train.columns[[0,1,2,3,4,5,6,7]]
# Iterate over given columns only 
for column in [1,3]:
   for row in range(len(X_train)): 
     if (X_train.iloc[row, column-1] == y_train.iloc[row]):
       pass
     else:
       X_train.iloc[row, column] = 1 - X_train.iloc[row, column]
       #  print(col_label[column], X_train.iloc[row, 0],X_train.iloc[row, column-1], X_train.iloc[row, column])

In [ ]:
# # only apply to training set
# col_label = df_pred.columns[[0,1,2,3,4,5,6,7,8]]
# # Iterate over given columns only 
# for column in [2,4,6,8]:
#    for row in range(len(df_pred)): 
#      if (df_pred.iloc[row, column-1] == df_pred.iloc[row, 0]):
#        pass
#      else:
#        df_pred.iloc[row, column] = 1 - df_pred.iloc[row, column]
#        #  print(col_label[column], df_pred.iloc[row, 0],df_pred.iloc[row, column-1], df_pred.iloc[row, column])


# df_pred = df_pred.drop(["ResNet_label","EffNet_label","Inception_label","VGG19_label"] , axis=1)

In [ ]:
# X_train = X_train.drop(["ResNet_label","EffNet_label","Inception_label","VGG19_label"] , axis=1)
# X_test = X_test.drop(["ResNet_label","EffNet_label","Inception_label","VGG19_label"] , axis=1)

X_train = X_train.drop(["Inception_label","VGG19_label"] , axis=1)
X_test = X_test.drop(["Inception_label","VGG19_label"] , axis=1)

In [ ]:
# """
# to test combination of models
# """
# # df_pred = df_pred.drop(["EffNet_pred"] , axis=1)  ## "ResNet_pred","EffNet_pred","Inception_pred","VGG19_pred"
# X_train = X_train.drop(["ResNet_pred","EffNet_pred"] , axis=1)
# X_test = X_test.drop(["ResNet_pred","EffNet_pred"] , axis=1)

## Random Forest train/test

In [ ]:
rfc = None
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predictions = rfc.predict(X_test)  

pd.DataFrame(rfc_predictions, index=X_test.index, columns=["test_pred"])

# Evaluate train/test

In [ ]:
rfc_f1_macro = f1_score(y_test, rfc_predictions, average = 'macro')
rfc_f1_macro

In [ ]:
conf_mat = confusion_matrix(y_test, rfc_predictions)
cm_display = ConfusionMatrixDisplay(conf_mat, display_labels=['crystal', 'no_crystal']).plot()

In [ ]:
class_rep = classification_report(y_test, rfc_predictions, 
                                  labels = ['crystal', 'no_crystal'], 
                                  digits=2, 
                                  zero_division='warn')
print(class_rep)

In [ ]:
# ### To run ROC maybe first transform categorical features into [0,1]
# fpr, tpr, _ = roc_curve(y_test, rfc_predictions, pos_label=rfc.classes_[1])
# roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

## Predict on new data

In [ ]:
### Create DataFrame with test images to be populated by the prediction of each model 

folder_path = "/gdrive/My Drive/Final_Project_CrystalsFirst/Model/labels/new_data/"
img_files = {}
for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".JPG")):
      img_files[filename] = ()

X_new = pd.DataFrame.from_dict(img_files, orient="index")

In [ ]:
IMG_SIZE = 224
classes = ['crystal', 'no_crystal']
folder_path = "/gdrive/My Drive/Final_Project_CrystalsFirst/Model/labels/new_data/"

# models = [new_model_ResNet, new_model_EffNet, new_model_Inception , new_model_VGG19]
models = [new_model_Inception , new_model_VGG19]

# models_names = ["ResNet", "EffNet", "Inception", "VGG19"]
models_names = ["Inception", "VGG19"]

i=0
for new_model in models:
  predictions = {}
  for filename in os.listdir(folder_path):
      if (filename.endswith(".jpg") or filename.endswith(".JPG")):
        img = keras.preprocessing.image.load_img(folder_path+filename, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)    
        pred = new_model.predict(img_array)
        predictions[filename] = (classes[np.argmax(pred)], (np.max(pred))) 
  # X_new[models_names[i]+ "_file"]   = predictions.keys()
  X_new[models_names[i] + "_label"] = ([t[0] for t in predictions.values()])
  X_new[models_names[i] + "_pred"]  = ([t[1] for t in predictions.values()])
  i=i+1

In [ ]:
X_new.head(2)

In [ ]:
# # only apply to training set
# # col_label = X_new.columns[[0,1,2,3,4,5,6,7]]
# # Iterate over given columns only 
# for column in [1,3]:
#    for row in range(len(X_new)): 
#      if (X_new.iloc[row, column-1] == "no_crystal"):
#        pass
#      else:
#        X_new.iloc[row, column] = 1 - X_new.iloc[row, column]
#        #  print(col_label[column], X_new.iloc[row, 0],X_new.iloc[row, column-1], X_new.iloc[row, column])


# X_new = X_new.drop(["Inception_label","VGG19_label"] , axis=1)
# # X_new = X_new.drop(["ResNet_label","EffNet_label","Inception_label","VGG19_label"] , axis=1)

In [ ]:
X_new = X_new.drop(["Inception_label","VGG19_label"] , axis=1)
X_new.head(2)

In [ ]:
""" X_New must have same number of features as X_test"""
# X_new = X_new.drop(["ResNet_pred","EffNet_pred"] , axis=1)

rfc_predictions = rfc.predict(X_new)  

In [ ]:
final_df = pd.DataFrame(rfc_predictions, index=X_new.index, columns=["Predictions"])
final_df